In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from training import train
from eval_erisk import evaluate
from IPython.display import display, Markdown
from itertools import product

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
experiment_values = {
    "cnn_model": [True],
    "oversample": [False, True],
    "feat_window_size": [20],
    "normalize": [False],
    "discretize": [False, True],
    "exclude_feats": [["sentiment", "methods", "terms", "instruments", "reasons"]],
    "eval_window_size": [100],
    "max_len": [1000],
    "batch_size": [32, 128]
}

In [5]:
logger("Experiment report: {}".format(experiment_values))

Experiment report: {'cnn_model': [True], 'oversample': [False, True], 'feat_window_size': [1, 3, 10, 20], 'normalize': [False, True], 'discretize': [False, True], 'exclude_feats': [[], ['methods', 'terms', 'instruments', 'reasons'], ['sentiment', 'methods', 'terms', 'instruments', 'reasons']], 'eval_window_size': [10, 100, 1000], 'max_len': [1000]}


In [6]:
def do_experiment(params):
    logger("Starting experiment {}".format(params))
    windowfy(window_size=params["feat_window_size"], is_oversample=params["oversample"])
    featurize(calculate_feats=True, normalize=params["normalize"], discretize=params["discretize"],
                      exclude_feats=params["exclude_feats"])
    train(cnn_model=params["cnn_model"], maxlen=params["max_len"], batch_size=params["batch_size"])
    evaluate(params["eval_window_size"])
    logger("Finished experiment {}".format(params))
    
def traverse(d):
    K,V = zip(*d.items())
    for v in product(*(v if isinstance(v,list) else traverse(v) for v in V)):
        yield dict(zip(K,v))

In [2]:
experiment_values = {
    "cnn_model": True,
    "oversample": False,
    "feat_window_size": 1000,
    "normalize": False,
    "discretize": False,
    "exclude_feats": [],
    "eval_window_size": 1000,
    "max_len": 1000,
    "batch_size": 2
}

## Experiments

In [ ]:
#preprocess()

In [5]:
experiments_generator = traverse(experiment_values)

for i in experiments_generator:
    display(Markdown("#### Experiment {}".format(i)))
    do_experiment(i)
    
logger("Finished experiments")

#### Experiment {'cnn_model': True, 'oversample': False, 'feat_window_size': 1, 'normalize': False, 'discretize': False, 'exclude_feats': [], 'eval_window_size': 10, 'max_len': 1000}

Starting experiment {'cnn_model': True, 'oversample': False, 'feat_window_size': 1, 'normalize': False, 'discretize': False, 'exclude_feats': [], 'eval_window_size': 10, 'max_len': 1000}
Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Finished windowfying
Featurizing calculate_feats=True, normalize=False, discretize=False, discretize_size=10, exclude_feats=[]
Initialized numpy random and tensorflow random seed at 42
Starting training with cnn_model=True and maxlen=1000
Generating embeddings


MemoryError: Unable to allocate 31.7 GiB for an array with shape (170433, 50000) and data type int32